# Object GUI

In [14]:
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import clear_output
import os
import glob
import hickle
import importlib
import codecs


# create initial widget
module_sel = widgets.Select()
module_sel.options = []
module_sel.rows=10

object_sel = widgets.Select()
object_sel.option = []
object_sel.rows=10

text_display = widgets.VBox([])

object_import = None

class ObjectGUI:
    def __init__(self, *args, **kwargs):
        if bool(glob.glob("config.hkl")):
            data = hickle.load("config.hkl")
            for (k, v) in data.items():
                setattr(self, k, v)
            # set up
            module_sel.options = self.module_list.keys()
            for module in module_sel.options:
                importlib.import_module(module)
            if module_sel.value == None:
                object_sel.option = []
            else:
                object_sel.options = self.module_list[module_sel.value]
                object_sel.value = self.module_list[module_sel.value][0]
                
                # update argList
                module_import = importlib.import_module(module_sel.value)
                global object_import
                object_import = getattr(module_import, object_sel.value)
                argsList = getattr(object_import, 'argsList', 'None')
                text_list = []
                if argsList != 'None':
                    for key in argsList:
                        if type(key[1]) == str:
                            a = widgets.Text(value = key[1],
                                         placeholder='Type something',
                                         description= key[0],
                                         disabled=False)
                            text_list += [a]
                        elif type(key[1]) == int:
                            a = widgets.IntText(
                                        value=key[1],
                                        description=key[0],
                                        disabled=False)
                            text_list += [a]
                        elif type(key[1]) == bool:
                            a = widgets.Checkbox(
                                        value=key[1],
                                        description=key[0],
                                        disabled=False)
                            text_list += [a]
                        elif type(key[1]) == list:
                            if type(key[1][0]) == int:
                                a = widgets.Text(value = str(key[1]),
                                             placeholder='list int',
                                             description= key[0],
                                             disabled=False)
                            elif type(key[1][0]) == float:
                                a = widgets.Text(value = str(key[1]),
                                             placeholder='list float',
                                             description= key[0],
                                             disabled=False)
                            text_list += [a]
                        else:
                            a = widgets.Text(value = str(key[1]),
                                         placeholder='Type something',
                                         description= key[0],
                                         disabled=False)
                            text_list += [a]
                text_display.children=tuple(text_list)
            
            os.chdir(self.work_directory)
            
        else:
            self.module_list = {}
            self.work_directory = os.getcwd()
            self.config_directory = os.getcwd()
            self.work_directory_list = []
    
    def save(self):
        tosave = {}
        for (k, v) in self.__dict__.items():
            tosave[k] = v
        os.chdir(self.config_directory)
        hickle.dump(tosave, "config.hkl", mode="w")
        os.chdir(self.work_directory)


# create setting object        
config = ObjectGUI()
        
# Create new FileChooser:
# Path: current directory
# Show hidden files: no
fdialog = FileChooser(
    config.work_directory,
    filename='',
    title='',
    change_desc = 'Select...',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

title_fdialog = widgets.HTML(value = f"<font color='b'><font size=5><b>{'Working Directory'}")

# Callback func
def change_directory(chooser):
    config.work_directory = chooser.selected_path
    os.chdir(chooser.selected_path)
    
fdialog.register_callback(change_directory)

choose_wd = widgets.VBox([title_fdialog,fdialog])

In [15]:
#processLevel
import DataProcessingTools as DPT

level_input = widgets.Text(value='',
                         placeholder='level',
                         description='Level:',
                         disabled=False)
cmd_input = widgets.Textarea(value='',
                         placeholder='CMD',
                         description='CMD:',
                         disabled=False)
process_level_button = widgets.Button(description='Process Level')
process_level_out = widgets.Output()

def on_process_level_clicked(_):
    with process_level_out:
        level = level_input.value
        cmd = cmd_input.value
        dirs, data = DPT.levels.processLevel(level,cmd)
        
        clear_output(True)
        display(dirs)
        display(data)
        
title_processLevel = widgets.HTML(value = f"<font color='b'><font size=5><b>{'Process Level'}")
process_level_button.on_click(on_process_level_clicked)            

processLevel = widgets.VBox([title_processLevel,widgets.HBox([level_input,cmd_input]),process_level_button,process_level_out])

display(widgets.HBox([choose_wd,processLevel]))

##  Module Directory

In [16]:
fdialog2 = FileChooser(
    os.getcwd(),
    filename='',
    title='',
    change_desc = 'Add...',
    show_hidden=False,
    select_default=True,
    use_dir_icons=True
)

display(fdialog2)

FileChooser(path='/Users/chris/Downloads/picasso-misc/20181105/session01', filename='', show_hidden='False')

In [17]:
# Callback func
def search_module(chooser):
    cw = os.getcwd()
    os.chdir(chooser.selected_path)
    module_name = os.path.basename(os.path.normpath(chooser.selected))
    class_list = []
    # look for Module name folder
    if bool(glob.glob(module_name)):
        os.chdir(module_name)
        if bool(glob.glob("__init__.py")):
            ini_file = open("__init__.py")
            for line in ini_file.readlines():
                # skip empty lines
                line = line.strip()
                if not line:  # line is blank
                    continue
                last_word = line.split()[-1]
                if(last_word[0].isupper()):
                    class_list += [last_word]
            ini_file.close()
    # change possible selections
    config.module_list[module_name] = class_list
    module_sel.options = config.module_list.keys()
    module_sel.value = module_name
    display(module_sel)
    os.chdir(cw)

# Register callback function
fdialog2.register_callback(search_module)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        module_name = change['new']
        object_sel.options = config.module_list[module_name]
        object_sel.value = config.module_list[module_name][0]
        display(object_sel)

module_sel.observe(on_change)

remove_button = widgets.Button(description='Remove')

def on_remove_button_clicked(_):
    module_name = module_sel.value
    del config.module_list[module_name]
    module_sel.options = config.module_list.keys()
    
    
remove_button.on_click(on_remove_button_clicked)

In [18]:
def on_select_object(change):
    global object_import
    if change['type'] == 'change' and change['name'] == 'value':
        object_name = change['new']
        module_import = importlib.import_module(module_sel.value)
        object_import = getattr(module_import, object_name)
        argsList = getattr(object_import, 'argsList', 'None')
        text_list = []
        if argsList != 'None':
            for key in argsList:
                if type(key[1]) == str:
                    a = widgets.Text(value = key[1],
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
                elif type(key[1]) == int:
                    a = widgets.IntText(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                elif type(key[1]) == bool:
                    a = widgets.Checkbox(
                                value=key[1],
                                description=key[0],
                                disabled=False)
                    text_list += [a]
                elif type(key[1]) == list:
                    if type(key[1][0]) == int:
                        a = widgets.Text(value = str(key[1]),
                                     placeholder='list int',
                                     description= key[0],
                                     disabled=False)
                    elif type(key[1][0]) == float:
                        a = widgets.Text(value = str(key[1]),
                                     placeholder='list float',
                                     description= key[0],
                                     disabled=False)
                    text_list += [a]
                else:
                    a = widgets.Text(value = str(key[1]),
                                 placeholder='Type something',
                                 description= key[0],
                                 disabled=False)
                    text_list += [a]
        text_display.children=tuple(text_list)

object_sel.observe(on_select_object)

In [19]:
proceed_button = widgets.Button(description='Proceed')
proceed_out = widgets.Output()
style = {'description_width': 'initial'}
object_name = widgets.Text(value = None,
                           placeholder='Type something',
                           description= 'Object Name: ',
                           disabled=False,
                           style=style)

create_object_list = {}
object_list = widgets.Select()
object_list.options = create_object_list.keys()
object_list.rows=10

plot_select = widgets.SelectMultiple(
    options=[],
    disabled=False
)

file_upload = widgets.FileUpload(accept='',multiple=False,disabled = True)        

def find_content(change):
    content = change.owner.data[0]
    content = codecs.decode(content, encoding="utf-8")
    content = list(content.split('\n')) 
    config.work_directory_list = content
    with proceed_out:
        proceed_out.clear_output()
        display(content)

file_upload.observe(find_content, 'value')

process_option = widgets.Dropdown(options=['Current directory', 'All subdirectories', 'Select list of directories…'],
                                  value='Current directory',
                                  description='Create Object in: ',
                                  disabled=False,
                                  style=style)

def on_change_process_option(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'Select list of directories…':
            file_upload.disabled = False
        else:
            file_upload.disabled = True

process_option.observe(on_change_process_option)

def on_proceed_button_clicked(_):
    arg = text_display.children
    updated_arg_list = {}
    for a in arg:
        if type(a) == widgets.widget_string.Text:
            if a.placeholder == 'list int':
                updated_arg_list[a.description]=list(map(int, (a.value)[1:-1].split(",")))
            elif a.placeholder == 'list float':
                updated_arg_list[a.description]=list(map(float, (a.value)[1:-1].split(",")))
            else:
                updated_arg_list[a.description]=a.value
        else:
            updated_arg_list[a.description]=a.value

    with proceed_out:
        proceed_out.clear_output()
        name_input = object_name.value
        if process_option.value == 'Current directory':
            object_create = object_import(**updated_arg_list)
        elif process_option.value == 'All subdirectories':
            object_create = DPT.objects.processDirs(None, object_import,**updated_arg_list)
        elif process_option.value == 'Select list of directories…':
            object_create = DPT.objects.processDirs(config.work_directory_list, object_import,**updated_arg_list)
        create_object_list[name_input]= object_create
        object_list.options = create_object_list.keys()
        plot_select.options = create_object_list.keys()
        object_list.value = name_input
            
proceed_button.on_click(on_proceed_button_clicked)


title_arg = widgets.HTML(value = f"<font color='b'><font size=5>{'Constructor Options'}")
title_module = widgets.HTML(value = f"<font color='b'><font size=5>{'Module'}")
title_object = widgets.HTML(value = f"<font color='b'><font size=5>{'Object'}")
title_create = widgets.HTML(value = f"<font color='b'><font size=5>{'Object Creation'}")
display(widgets.HBox([widgets.VBox([title_module,module_sel,remove_button]),
                      widgets.VBox([title_object,object_sel]),
                      widgets.VBox([title_arg,text_display]),
                      widgets.VBox([title_create,object_name,widgets.HBox([process_option,file_upload]),proceed_button,proceed_out])]))

## Object List

In [20]:
attributes = {}
attributes_list = widgets.Select()
attributes_list.options = []
attributes_list.rows=10

attributes_out = widgets.Output()

def on_change_object_list(change):
    if change['type'] == 'change' and change['name'] == 'value':
        object_name = change['new']
        object_content = create_object_list[object_name]
        global attributes
        attributes = object_content.__dict__
        temp_list = []
        for key in attributes:
            temp_list += [key + '  ' + str(type(attributes[key]))]
        attributes_list.options = temp_list
        attributes_list.value = temp_list[0]
        with attributes_out:
            clear_output(True)
            attribute_name = temp_list[0].split()[0]
            attributes_content = attributes[attribute_name]
            display(attributes_content)

object_list.observe(on_change_object_list)

def on_change_attributes_list(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with attributes_out:
            clear_output(True)
            attribute_name = change['new'].split()[0]
            attributes_content = attributes[attribute_name]
            display(attributes_content)

attributes_list.observe(on_change_attributes_list)
display(widgets.HBox([object_list,attributes_list,attributes_out]))

remove_object_button = widgets.Button(description='Remove')

def on_remove_object_button_clicked(_):
    module_name = object_list.value
    del create_object_list[module_name]
    object_list.options = create_object_list.keys()
    plot_select.options = create_object_list.keys()
    
    
remove_object_button.on_click(on_remove_object_button_clicked)
display(remove_object_button)

Button(description='Remove', style=ButtonStyle())

## Plot

In [14]:
%matplotlib widget
import ipympl
import numpy as np
import matplotlib.pyplot as plt

plotobjs = None
index = 0
plotopts = None
plotopt_widget = {}
plotopt_sel_idx = None
numEvents = 100_000
plotopt_sel_idx = None
plotopt_sel_result = widgets.VBox([])
setting_widget = widgets.VBox(tuple(plotopt_widget.keys()))

link_xaxes = widgets.Text(value='',
                         placeholder='Seperate by comma',
                         description='linkxaxes',
                         disabled=False)
link_yaxes = widgets.Text(value='',
                         placeholder='Seperate by comma',
                         description='linkyaxes',
                         disabled=False)

plot_button = widgets.Button(description='Plot')
display(widgets.HBox([plot_select,widgets.VBox([link_xaxes,link_yaxes]),plot_button]))
fig= plt.figure()
fig.set_facecolor((0.92, 0.92, 0.92))

def on_plot_button_clicked(_): 
    global plotobjs,index,plotopts,plotopt_widget
    
    # clear Previous plot
    setting_widget.children = tuple([])
    plotopt_sel_result.children=tuple([])
    index_show.value = str(0)
    for _ax in fig.axes:
        _ax.remove()
        
    plotobjs = []
    for name in plot_select.value:
        plotobjs += [create_object_list[name]]
        
    linkxaxes = None
    linkyaxes = None
    if link_xaxes.value != '':
        value_x = list(link_xaxes.value.split(',')) 
        linkxaxes = list(map(int, value_x))
    if link_yaxes.value != '':
        value_y = list(link_yaxes.value.split(',')) 
        linkyaxes = list(map(int, value_y))
        
    if linkxaxes is None:
        linkxaxes = range(len(plotobjs))
    if linkyaxes is None:
        linkyaxes = range(len(plotobjs))
    
    plotopt_widget = {}
    plotopts = None
    index = 0
    if hasattr(plotobjs, '__getitem__'):
        plotobjs = plotobjs
    else:
        plotobjs = [plotobjs]
    
    plotopts = [plotobj.plot(getPlotOpts=True) for plotobj in plotobjs]
    cols = 1
    rows = np.ceil(len(plotobjs)/cols)
    
    for (i, a) in enumerate(plotopts):
        widget = widgets.Dropdown(options=['Select...']+list(a.keys()),
                                  value='Select...',
                                  description="Plot Setting",
                                  disabled=False)
        plotopt_widget[widget] = i
        widget.observe(select_setting)
    setting_widget.children=tuple(plotopt_widget.keys())
    
    global numEvents
    numEvents = 100_000
    for (i, plotobj) in enumerate(plotobjs):
        if linkxaxes[i] < i:
            sharex = fig.axes[linkxaxes[i]]
        else:
            sharex = None
        if linkyaxes[i] < i:
            sharey = fig.axes[linkxaxes[i]]
        else:
            sharey = None
        
        ax = fig.add_subplot(rows, cols, i+1, sharex=sharex,
                              sharey=sharey)
        nn, newIdx = plotobj.plot(index, getNumEvents=True)
        numEvents = min(numEvents, nn)
        plotobj.plot(index, ax=ax, **plotopts[i])
        if newIdx is not None and newIdx != index:
            index_show.value = str(newIdx)
            updateIndex()
            
plot_button.on_click(on_plot_button_clicked)

pre_button = widgets.Button(description='Pre')
next_button = widgets.Button(description='Next')
index_show = widgets.Text(value = '0',
                          placeholder='Index',
                          description= "",
                          disabled=False)

def updateIndex(wdgt):
    global plotobjs,plotopts
    update_index(int(wdgt.value))
    wdgt.value = str(index)
    for _ax in fig.axes:
        _ax.clear()
    for (i, plotobj) in enumerate(plotobjs):
        plotobj.plot(index, ax=fig.axes[i], **plotopts[i])
        
def update_index(new_index):
    global index
    if 0 <= new_index < numEvents:
        index = new_index
    else:
        index = index

def gonext(_):
    update_index(index+1)
    index_show.value = str(index)
    plot()

def goprev(_):
    update_index(index-1)
    index_show.value = str(index)
    plot()

def plot():
    global plotobjs,plotopts
    for _ax in fig.axes:
        _ax.clear()
    for (i, plotobj) in enumerate(plotobjs):
        plotobj.plot(index, ax=fig.axes[i], **plotopts[i])

def select_setting(change):
    if change['type'] == 'change' and change['name'] == 'value':

        widget = []
        if change['new'] != 'Select...':
            index = plotopt_widget[change.owner]
            new = change['new']

            for (k,v) in plotopt_widget.items():
                if v != index:
                    k.value = 'Select...'

            global plotopt_sel_idx
            plotopt_sel_idx = index
            plotopt_sel = plotopts[index]
            v = plotopt_sel[new]
            if isinstance(v, bool):
                w = widgets.Checkbox(value=v,
                                     description=new,
                                     disabled=False)
                w.observe(update_Checkbox)
                widget += [w]
            elif isinstance(v, dict):
                for key in v:
                    w = widgets.Text(value=str(v[key]),
                                     placeholder=new,
                                     description=key,
                                     disabled=False)
                    w.on_submit(update_dict)
                    widget += [w]
            elif isinstance(v, int):
                w = widgets.Text(value = str(v),
                                 placeholder='',
                                 description=new,
                                 disabled=False)
                w.on_submit(update_IntText)
                widget += [w]
            elif isinstance(v, float):
                w = widgets.Text(value = str(v),
                                 placeholder='',
                                 description=new,
                                 disabled=False)
                w.on_submit(update_floatText)
                widget += [w]
            elif isinstance(v, DPT.objects.ExclusiveOptions):
                w = widgets.Dropdown(options=v.options,
                                     value=v.options[v.checked],
                                     description=new,
                                     disabled=False)
                w.observe(update_ExclusiveOptions)
                widget += [w]
            else:
                w = widgets.Text(value = str(v),
                     placeholder='',
                     description=new,
                     disabled=False)
                w.on_submit(update_elseText)
                widget += [w]
        plotopt_sel_result.children=tuple(widget)

def update_ExclusiveOptions(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global plotopts
        p = plotopts[plotopt_sel_idx][change.owner.description]
        p.select(change['new'])
        update_level()

def update_IntText(wdgt):
    plotopts[plotopt_sel_idx][wdgt.description] = int(wdgt.value)
    update_level()

def update_Checkbox(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plotopts[plotopt_sel_idx][change.owner.description] = change['new']
        update_level()

def update_floatText(wdgt):
    plotopts[plotopt_sel_idx][wdgt.description] = float(wdgt.value)
    update_level()

def update_dict(wdgt):
    plotopts[plotopt_sel_idx][wdgt.placeholder][wdgt.description] = float(wdgt.value)
    update_level()

def update_elseText(wdgt):
    plotopts[plotopt_sel_idx][wdgt.description] = str(wdgt.value)
    update_level()

def update_level():
    # crazy high intial values so that the new value is always lower
    global plotobjs,plotopts,numEvents
    num_events = 100_000
    newIdx = 100_000
    for (plotobj, plotopt) in zip(plotobjs, plotopts):
        nn, _newIdx = plotobj.plot(index, getNumEvents=True, **plotopt)
        num_events = min(num_events, nn)
        newIdx = min(newIdx, _newIdx)
    numEvents = num_events
    index_show.value = str(newIdx)
    updateIndex(index_show)

index_show.on_submit(updateIndex)
pre_button.on_click(goprev)
next_button.on_click(gonext)
display(widgets.HBox([pre_button,index_show,next_button]))
display(widgets.HBox([setting_widget,plotopt_sel_result]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot by PanGUI with New Window

In [ ]:
import PanGUI
plot_button_pangui = widgets.Button(description='PanGUI Plot')

def on_plot_button_pangui_clicked(_):
    PanGUI.create_window(create_object_list[object_list.value])
            
plot_button_pangui.on_click(on_plot_button_pangui_clicked)  

display(plot_button_pangui)

## Settings

In [9]:
config_button = widgets.Button(description='Save configurations')
config_out = widgets.Output()

def on_config_button_clicked(_):
    with config_out:
        clear_output(True)
        config.save()
        display('Saved')
            
config_button.on_click(on_config_button_clicked)            
display(config_button)
display(config_out)

Button(description='Save configurations', style=ButtonStyle())

Output()